In [1]:
import importlib

import indra_gpt.chat_curate.chat_curate
importlib.reload(indra_gpt.chat_curate.chat_curate)
from indra_gpt.chat_curate.chat_curate import *

import indra_gpt.chat_curate.eval
importlib.reload(indra_gpt.chat_curate.eval)
from indra_gpt.chat_curate.eval import (curation_statement_evidence_pair_comparison_json, 
                                        curation_statement_comparison_json)

import pickle
from pathlib import Path


/Users/thomaslim/miniconda3/envs/indra_gpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: [2025-07-01 09:30:00] httpx - HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


In [2]:
RESULTS_DIR = Path.cwd().joinpath("output/curated_statements_sample_100")
if not RESULTS_DIR.exists():
    RESULTS_DIR.mkdir(parents=True)


In [3]:
import random
import pickle
from pathlib import Path
from tqdm import tqdm
from indra.sources import indra_db_rest

# Step 1: Get all curations from the INDRA DB
print("📥 Fetching all curations from INDRA DB...")
curations = indra_db_rest.get_curations()

# Step 2: Extract unique statement hashes from curations
print("🧹 Extracting unique statement hashes...")
unique_hashes = list({c['pa_hash'] for c in curations if 'pa_hash' in c})

# Step 3: Randomly sample up to 100 curated statement hashes
sample_size = min(100, len(unique_hashes))
sampled_hashes = random.sample(unique_hashes, sample_size)

# Step 4: Fetch full statements by hash (including all evidences)
print(f"🔍 Fetching {sample_size} curated statements from INDRA DB...")
stmt_proc = indra_db_rest.get_statements_by_hash(sampled_hashes, ev_limit=100)
curated_statements = stmt_proc.statements

# Step 5: Save to pickle
output_path = RESULTS_DIR / "statements.pkl"

with open(output_path, "wb") as f:
    pickle.dump(curated_statements, f)

print(f"✅ Saved {len(curated_statements)} curated statements to: {output_path}")


INFO: [2025-07-01 09:30:00] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list
INFO: [2025-07-01 09:30:00] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 09:30:00] indra.sources.indra_db_rest.util - data: None


📥 Fetching all curations from INDRA DB...


INFO: [2025-07-01 09:30:02] indra_db_rest.query_processor - Retrieving statements that have hash -10558769985488922, -12064636854697961, -13295983281851382, -14213213083155331, -16253859176692871, -16488861539572024, -16507077386690032, -16547197524937149, -16614904339675969, -18863224253297223, -19358276560732315, -19540873943256728, -19594858394048257, -20095856845760100, -20155356540515066, -20905000153811208, -22129299903885771, -24407073837268975, -25167674544775862, -25685577301218461, -25687800924410749, -26040559113565892, -26186913902459663, -29144964812931735, -29611002097870603, -29615932207517012, -3035906237295811, -31640842146491461, -31649991562866730, -31734835008003753, -33584033074123684, -33633204848848179, -35980800557648489, -36016319529798526, -4259562148975527, -4289213873112161, -4820006348997044, -627992662320746, -6287275938202915, -7189243357890503, -8918741695867224, -895298493819940, -980605458670407, 10900723066461018, 12039417786486187, 12852949112266658,

🧹 Extracting unique statement hashes...
🔍 Fetching 100 curated statements from INDRA DB...
✅ Saved 89 curated statements to: /Users/thomaslim/gyorilab/indra_gpt/output/curated_statements_sample_100/statements.pkl


In [ ]:
statements = pickle.load(
    open(RESULTS_DIR.joinpath("statements.pkl"), "rb")
)
statements = statements
models = ['gpt-4o-mini']
for model in models:
    llm_curations_binary = chat_curate_stmts(statements, 
                                    n_evidence_to_curate=5,
                                    decision_threshold=0.5,
                                    binary_classification=True,
                                    #ignore_tags=['incorrect'],
                                    n_fewshot_examples=2,
                                    get_ex_per_tag=True,
                                    schema_output_mode=True,
                                    model=model)
    llm_curations_multiclass = chat_curate_stmts(statements,
                                    n_evidence_to_curate=5,
                                    decision_threshold=0.5,
                                    binary_classification=False,
                                    ignore_tags=['incorrect'],
                                    n_fewshot_examples=2,
                                    get_ex_per_tag=True,
                                    schema_output_mode=True,
                                    model=model)

    if not RESULTS_DIR.joinpath(model).exists():
        RESULTS_DIR.joinpath(model).mkdir(parents=True)
        
    with open(RESULTS_DIR / model / "chat_curation_binary_classification.json", "w") as f:
        json.dump(llm_curations_binary, f, indent=2)
    with open(RESULTS_DIR / model / "chat_curation_multiclass_classification.json", "w") as f:
        json.dump(llm_curations_multiclass, f, indent=2)


Curating statements with chat curation:   0%|          | 0/89 [00:00<?, ?it/s]INFO: [2025-07-01 09:30:46] indra_gpt.chat_curate.chat_curate - Prompt generation took 1.68 seconds
09:30:46 - LiteLLM:INFO: utils.py:2931 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-07-01 09:30:46] LiteLLM - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-07-01 09:30:48] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:30:48 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO: [2025-07-01 09:30:48] LiteLLM - Wrapper: Completed Call, calling success_handler
09:30:48 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-01 09:30:48] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
09:30:48 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



RateLimitError: litellm.RateLimitError: RateLimitError: OpenAIException - You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
for model in models:
    with open(RESULTS_DIR / model / "chat_curation_binary_classification.json", "r") as f:
        chat_curation_binary_classification = json.load(f)
    with open(RESULTS_DIR / model / "chat_curation_multiclass_classification.json", "r") as f:
        chat_curation_multiclass_classification = json.load(f)

    eval_stmt_ev_pair_binary = curation_statement_evidence_pair_comparison_json(chat_curation_binary_classification)
    with open(RESULTS_DIR / model / "eval_stmt_ev_pair_chat_curation_binary_classification.json", "w") as f:
        json.dump(eval_stmt_ev_pair_binary, f, indent=2)

    eval_stmt_binary = curation_statement_comparison_json(chat_curation_binary_classification)
    with open(RESULTS_DIR / model / "eval_stmt_chat_curation_binary_classification.json", "w") as f:
        json.dump(eval_stmt_binary, f, indent=2)

    eval_stmt_ev_pair_multi = curation_statement_evidence_pair_comparison_json(chat_curation_multiclass_classification)
    with open(RESULTS_DIR / model / "eval_stmt_ev_pair_chat_curation_multiclass_classification.json", "w") as f:
        json.dump(eval_stmt_ev_pair_multi, f, indent=2)

    eval_stmt_multi = curation_statement_comparison_json(chat_curation_multiclass_classification)
    with open(RESULTS_DIR / model / "eval_stmt_chat_curation_multiclass_classification.json", "w") as f:
        json.dump(eval_stmt_multi, f, indent=2)


Processing LLM curations:   0%|          | 0/93 [00:00<?, ?it/s]INFO: [2025-06-30 21:48:57] indra_gpt.chat_curate.eval - Error processing evidence curation: 'explanation'skipping this curation
INFO: [2025-06-30 21:48:57] indra_gpt.chat_curate.eval - Error processing evidence curation: 'explanation'skipping this curation
INFO: [2025-06-30 21:48:57] indra_gpt.chat_curate.eval - Error processing evidence curation: 'explanation'skipping this curation
INFO: [2025-06-30 21:48:57] indra_gpt.chat_curate.eval - Error processing evidence curation: 'explanation'skipping this curation
INFO: [2025-06-30 21:48:57] indra_gpt.chat_curate.eval - Error processing evidence curation: 'explanation'skipping this curation
INFO: [2025-06-30 21:48:57] indra_gpt.chat_curate.eval - Error processing evidence curation: 'explanation'skipping this curation
INFO: [2025-06-30 21:48:57] indra_gpt.chat_curate.eval - Error processing evidence curation: 'explanation'skipping this curation
INFO: [2025-06-30 21:48:57] indra

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

for model in models:
    print(f"Model: {model}")
    for eval_strat in ['binary_classification', 'multiclass_classification']:
        with open(RESULTS_DIR / model / f"eval_stmt_chat_curation_{eval_strat}.json", "r") as f:
            res = json.load(f)
        preds = [res_item['llm_overall_prediction'] for res_item in res]
        labels = [res_item['indra_curation'] for res_item in res]
        acc = accuracy_score(labels, preds)
        print(f"{eval_strat} - Accuracy: {acc:.4f}")


Model: gpt-4o-mini
binary_classification - Accuracy: 0.8387
multiclass_classification - Accuracy: 0.7742


In [ ]:
eval_stmt_binary


[{'eng_stmt': 'BAX activates cell death.',
  'pa_hash': -27178959702299212,
  'llm_overall_prediction': 'correct',
  'indra_curation': 'correct'},
 {'eng_stmt': 'MTOR activates translation.',
  'pa_hash': -19509266375111713,
  'llm_overall_prediction': 'correct',
  'indra_curation': 'correct'},
 {'eng_stmt': 'ACE2 inhibits Angiotensin-2.',
  'pa_hash': 31183949835825955,
  'llm_overall_prediction': 'correct',
  'indra_curation': 'incorrect'},
 {'eng_stmt': 'RELA binds RELA.',
  'pa_hash': -920869035786162,
  'llm_overall_prediction': 'correct',
  'indra_curation': 'incorrect'},
 {'eng_stmt': 'EGF activates RAS.',
  'pa_hash': 24957834887591847,
  'llm_overall_prediction': 'correct',
  'indra_curation': 'correct'},
 {'eng_stmt': 'Cocaine activates NLRP1.',
  'pa_hash': -33418178478117880,
  'llm_overall_prediction': 'incorrect',
  'indra_curation': 'incorrect'},
 {'eng_stmt': 'SOCS1 inhibits JAK.',
  'pa_hash': 35857458895716258,
  'llm_overall_prediction': 'correct',
  'indra_curation'

In [ ]:
# from sklearn.metrics import confusion_matrix
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Extract tags
# tags = [c['tag'] for c in curation_comp_json]
# predicted_tags = [c['predicted_tag'] for c in curation_comp_json]

# # Compute accuracy
# accuracy = np.mean([t == p for t, p in zip(tags, predicted_tags)])
# accuracy_bin = np.mean([t == p or (t != 'correct' and p == 'incorrect') for t, p in zip(tags, predicted_tags)])
# print(f"Accuracy: {accuracy:.2%}")
# print(f"Binary Accuracy: {accuracy_bin:.2%}")

# # Define tag label set
# tag_set = list(set(tags).union(set(predicted_tags)))

# # Compute confusion matrix
# cm = confusion_matrix(tags, predicted_tags, labels=tag_set)
# cm_df = pd.DataFrame(cm, index=tag_set, columns=tag_set)

# # Plot
# plt.figure(figsize=(12, 10))
# sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar=True, linewidths=0.5)
# plt.title("Confusion Matrix (Binary Classification - Model only predicts 'correct' or 'incorrect')")
# plt.xlabel("Predicted Label")
# plt.ylabel("True Label")
# plt.xticks(rotation=45, ha='right')
# plt.yticks(rotation=0)
# plt.tight_layout()
# plt.show()
